In [0]:
countries = [("USA", 10000, 20000), ("India", 1000, 1500), ("UK", 7000, 10000), ("Canada", 500, 700) ]
columns = ["Country","NumVaccinated","AvailableDoses"]
spark.createDataFrame(data=countries, schema = columns).write.format("delta").mode("overwrite").saveAsTable("silverTable")

In [0]:
%sql
DESCRIBE HISTORY silverTable

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2023-04-21T04:40:02.000+0000,3845443336354667,priya.ananthram@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2163243470721561),0421-024253-flf2xeso,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 4, numOutputBytes -> 4620)",null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql
select * from silverTable

Country,NumVaccinated,AvailableDoses
Canada,500,700
India,1000,1500
USA,10000,20000
UK,7000,10000


In [0]:
import pyspark.sql.functions as F
spark.read.format("delta").table("silverTable").withColumn("VaccinationRate", F.col("NumVaccinated") / F.col("AvailableDoses")) \
  .drop("NumVaccinated").drop("AvailableDoses") \
  .write.format("delta").mode("overwrite").saveAsTable("goldTable")

In [0]:
%sql
select * from goldTable

Country,VaccinationRate
Canada,0.7142857142857143
India,0.6666666666666666
USA,0.5
UK,0.7


In [0]:
%sql
ALTER TABLE silverTable SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
%sql
describe history silverTable

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2023-04-21T04:40:24.000+0000,3845443336354667,priya.ananthram@gmail.com,SET TBLPROPERTIES,"Map(properties -> {""delta.enableChangeDataFeed"":""true""})",null,List(2163243470721561),0421-024253-flf2xeso,0,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12
0,2023-04-21T04:40:02.000+0000,3845443336354667,priya.ananthram@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2163243470721561),0421-024253-flf2xeso,null,WriteSerializable,false,"Map(numFiles -> 4, numOutputRows -> 4, numOutputBytes -> 4620)",null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
# Insert new records
new_countries = [("Australia", 100, 3000)]
spark.createDataFrame(data=new_countries, schema = columns).write.format("delta").mode("append").saveAsTable("silverTable")

In [0]:
%sql
select * from silverTable

Country,NumVaccinated,AvailableDoses
Australia,100,3000
Canada,500,700
India,1000,1500
USA,10000,20000
UK,7000,10000


In [0]:
#Watermark the silvertable version
from pyspark.sql import functions as F

df = spark.sql('DESCRIBE HISTORY silverTable')\
        .groupby()\
        .agg(F.max('version'), F.min('version'), F.max('timestamp'), F.min('timestamp'))


display(df)
last_commit_version = df.collect()[0][0]
last_commit_version

In [0]:
%sql
-- update a record
UPDATE silverTable SET NumVaccinated = '11000' WHERE Country = 'USA'

num_affected_rows
1


In [0]:
%sql
-- delete a record
DELETE from silverTable WHERE Country = 'UK'

num_affected_rows
1


In [0]:
%sql
select * from silverTable

Country,NumVaccinated,AvailableDoses
USA,11000,20000
Australia,100,3000
Canada,500,700
India,1000,1500


In [0]:
spark.conf.set ('db.version',last_commit_version)

In [0]:
%sql 
-- view the changes
SELECT * FROM table_changes('silverTable',${db.version}) order by _commit_timestamp

Country,NumVaccinated,AvailableDoses,_change_type,_commit_version,_commit_timestamp
Australia,100,3000,insert,2,2023-04-21T04:41:01.000+0000
USA,10000,20000,update_preimage,3,2023-04-21T04:41:07.000+0000
USA,11000,20000,update_postimage,3,2023-04-21T04:41:07.000+0000
UK,7000,10000,delete,4,2023-04-21T04:41:12.000+0000


In [0]:
spark.read.format("delta").option("readChangeData", True).option("startingVersion", 2).table('silverTable')

changes_df = spark.read.format("delta").option("readChangeData", True).option("startingVersion", last_commit_version).table('silverTable')
display(changes_df)

Country,NumVaccinated,AvailableDoses,_change_type,_commit_version,_commit_timestamp
USA,10000,20000,update_preimage,3,2023-04-21T04:41:07.000+0000
USA,11000,20000,update_postimage,3,2023-04-21T04:41:07.000+0000
UK,7000,10000,delete,4,2023-04-21T04:41:12.000+0000
Australia,100,3000,insert,2,2023-04-21T04:41:01.000+0000


In [0]:
%sql
-- Collect only the latest version for each country
CREATE OR REPLACE TEMPORARY VIEW silverTable_latest_version as
SELECT * 
    FROM 
         (SELECT *, rank() over (partition by Country order by _commit_version desc) as rank
          FROM table_changes('silverTable', ${db.version})
          WHERE _change_type !='update_preimage')
    WHERE rank=1

In [0]:
%sql
select * from silverTable_latest_version

Country,NumVaccinated,AvailableDoses,_change_type,_commit_version,_commit_timestamp,rank
Australia,100,3000,insert,2,2023-04-21T04:41:01.000+0000,1
UK,7000,10000,delete,4,2023-04-21T04:41:12.000+0000,1
USA,11000,20000,update_postimage,3,2023-04-21T04:41:07.000+0000,1


In [0]:
%sql
-- Merge the changes to gold
MERGE INTO goldTable t USING silverTable_latest_version s ON s.Country = t.Country
        WHEN MATCHED AND s._change_type='update_postimage' THEN UPDATE SET VaccinationRate = s.NumVaccinated/s.AvailableDoses
        WHEN NOT MATCHED THEN INSERT (Country, VaccinationRate) VALUES (s.Country, s.NumVaccinated/s.AvailableDoses)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2,1,0,1


In [0]:
%sql
select * from goldTable

Country,VaccinationRate
Australia,0.03333333333333333
USA,0.55
Canada,0.7142857142857143
India,0.6666666666666666
UK,0.7


In [0]:
%sql
DROP TABLE silverTable;
DROP TABLE goldTable;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-3278532086564604> in <cell line: 1>()
      6     display(df)
      7     return df
----> 8   _sqldf = ____databricks_percent_sql()
      9 finally:
     10   del ____databricks_percent_sql

<command-3278532086564604> in ____databricks_percent_sql()
      3     import base64
      4     spark.sql(base64.standard_b64decode("RFJPUCBUQUJMRSBzaWx2ZXJUYWJsZQ==").decode())
----> 5     df = spark.sql(base64.standard_b64decode("RFJPUCBUQUJMRSBnb2xkVGFibGU=").decode())
      6     display(df)
      7     return df

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time